## Learn to classify

Sample using scikit-learn "Machine Learning in Python" to create efficient MAV code.

In this example a simple color classifier is learned from sample data.
The function "create_mask" creates masks for test images automaticaly
using HSV color filtering.

Add some images to the data folder.
 - Image file names should end with .jpg and be in the folder "test_footage" 

In [1]:
import cv2
import numpy as np
import glob
from random import randrange

#function for manual labeling of data images
def labeling(input_image):
    img = input_image.copy()
    pt1 = None; pt2 = None
    
    # Define a callback function for mouse events
    def draw_rectangle(event, x, y, flags, params):
        nonlocal pt1, pt2
        if event == cv2.EVENT_LBUTTONDOWN:
            pt1 = (x, y)#fix pt1
            pt2 = (x, y)#initial value for pt2
        elif event == cv2.EVENT_LBUTTONUP:
            pt2 = (x, y)#updating pt2 as we move the cursor
            cv2.rectangle(img, pt1, pt2, (255, 255, 255), -1)
            cv2.imshow('Input Image', img)
        elif event == cv2.EVENT_MOUSEMOVE:
            if flags & cv2.EVENT_FLAG_LBUTTON:
                pt2 = (x, y)#fix pt2
                img_copy = img.copy()
                cv2.rectangle(img_copy, pt1, pt2, (255, 255, 255), -1)
                cv2.imshow('Input Image', img_copy)

    # Create a window to display the input image
    cv2.namedWindow('Input Image')

    # Set the callback function for mouse events
    cv2.setMouseCallback('Input Image', draw_rectangle)

    # Display the input image and wait for the user to draw a rectangle
    cv2.imshow('Input Image', img)
    cv2.waitKey(0)

    # Create a black and white mask image with the same dimensions as the input image
    mask = np.zeros_like(input_image)
    cv2.rectangle(mask, pt1, pt2, (255, 255, 255), -1)

    # Destroy all windows
    cv2.destroyAllWindows()

    # Return the resulting black and white mask image
    return mask


In [2]:
#Function to create a black and white image which is white where the obstacles is found
#(based on color filtering in HSV)

def create_mask(image, color):
    image = cv2.imread(image)#creating an numpy array from the image
    #crop the image to only have the horizon left
    image = image[:, 80:180]
    
    #convert to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define the HSV range for each color
    if (color == "orange"):
        lower = np.array([0, 100, 100])
        upper = np.array([25, 255, 255])
    elif (color == "green"):
        lower = np.array([35, 80, 40])
        upper = np.array([70, 255, 100])
    elif (color == "white"):
        lower = np.array([0, 0, 100])
        upper = np.array([150, 40, 240])
        
    elif (color == "black"):
        lower = np.array([0, 0, 50])
        upper = np.array([125, 150, 125])
    else:#in case there is no one color for the obstacle, we can create the labels by hand
        result = labeling(image)#choose the zone where the obstacle is on the image
        cv2.destroyAllWindows()
        return result, image
    
    # Create and apply the mask to the image to create the mask image
    mask = cv2.inRange(hsv_image, lower, upper) #create mask
    result = cv2.bitwise_and(image, image, mask=mask)#apply mask
    result = cv2.threshold(result[:,:,2], 1, 255, cv2.THRESH_BINARY)[1] #either white or black
    result = cv2.cvtColor(result, cv2.COLOR_GRAY2RGB)#convert back to rgb
    
    return result, image


In [9]:
#####FETCH THE IMAGES###################
images_string = glob.glob("test_footage/*.jpg")
     
print(len(images_string))#check to see if it worked
print(type(images_string[1]))

17
<class 'str'>


In [10]:
#####CREATE THE IMAGE and LABELS (MASK) LIST############
images = []
labels = []
#create a mask for every image
for i in images_string:
    mask, img = create_mask(i, "forest")#dont forget to change the color
    labels.append(mask)
    images.append(img)


print(len(images))#check to see if it everything is fine
print(len(labels))#there should be the same number of images as labels
print(np.shape(labels[0]))#check the dimensions of the label

17
17
(520, 240, 3)


From all the images and all the masks, create one large vector containing all training samples. 
 - The input vector has three columns for Y U V
 - The output has the label, ground or non-ground
 
 _Tip: modify the code to add several adjacent pixels as inputs_

In [11]:
# Convert images to training data (vector)

X_vec = []
y_vec = []

maxfiles = 500
samples_per_image = 75000

for i in range(len(images)):
    maxfiles -= 1
    if maxfiles <=0:
            break
    img = images[i]
    msk = labels[i]
    h,w,d = img.shape
    
    #print('img=',f,'lbl=',lf,w,'x',h)

     # Color space of RAW Bebop Images
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img[:,:,1] = msk[:,:,0]

    for i in range(0,samples_per_image):
        
        x = randrange(2,w-3)
        y = randrange(4,h-2)

        # Pixels
        p = yuv[y,x]
        # Ground thruth
        m = int(msk[y,x,0])
        if m < 127:
            m = 0
        else:
            m=255

        # Y, U, V
        X_vec.append([int(p[0]),int(p[1]),int(p[2])])
        y_vec.append([m])
        

        

print('Dataset',len(X_vec), len(y_vec))

Dataset 1275000 1275000


Use 20% of the data as test data and 80% as training data

In [12]:
# Split training data and test data

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_vec, y_vec, test_size=0.2, stratify=y_vec, random_state=1)

print('Train',len(X_train), len(y_train))
print('Test',len(X_test), len(y_test))



Train 1020000 1020000
Test 255000 255000


Create a DecisionTreeClassifier, train and show the score.

In [13]:
# Train our model

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt = DecisionTreeClassifier(max_depth=2, random_state=0)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

score = accuracy_score(y_test, y_pred)
print('Sensitivity:',round(score,3))


Sensitivity: 0.869


### Export

The decision tree can be exported. Convert this to c-code to run it on the drone.

In [14]:
# Export values in the form of a decision tree

from sklearn.tree import export_text

text_representation = export_text(dt, feature_names=['Y','U','V'])

print(text_representation)

|--- V <= 155.50
|   |--- Y <= 144.50
|   |   |--- class: 0
|   |--- Y >  144.50
|   |   |--- class: 0
|--- V >  155.50
|   |--- U <= 96.50
|   |   |--- class: 255
|   |--- U >  96.50
|   |   |--- class: 0

